# Data Analysis

## View some data

In [1]:
from constants.data_constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from constants.real_time_constants import SLIDING_WINDOW_LEN, STRIDE
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset
from data.sliding_window import SlidingWindowDataset

## pure data
id_train_base_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train",
    name="id_train_dataset"
)
id_test_base_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation",
    name="id_test_dataset"
)
ood_test_base_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
    name="maestro_test_dataset"
)

## dataset that takes chunks of 120 tokens out of the above datasets
id_train_dataset = SlidingWindowDataset(
    base_dataset=id_train_base_dataset,
    name="id_train_dataset",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)
id_test_dataset = SlidingWindowDataset(
    base_dataset=id_test_base_dataset,
    name="id_test_dataset",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)   
ood_test_dataset = SlidingWindowDataset(
    base_dataset=ood_test_base_dataset,
    name="ood_test_dataset",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)




/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sample tokens:
Detected 0 bad samples
Sample tokens:
Detected 0 bad samples


In [2]:
import random
import pandas as pd
from utils.process_tokens import get_readable_events

events_by_dataset = {}
for name, dataset in [("id_train", id_train_dataset), ("id_test", id_test_dataset), ("ood_test", ood_test_dataset)]:
    random_idx = random.randint(0, len(dataset) - 1)
    input_ids = dataset[random_idx]['input_ids']
    if hasattr(input_ids, 'tolist'):
        input_ids = input_ids.tolist()
    try:
        events = get_readable_events(input_ids)
    except Exception as e:
        print(f"Error: {e}")
        print(f"Input IDs: {input_ids}")
        continue
    events_by_dataset[name] = events

min_events = min(len(events) for events in events_by_dataset.values())
num_rows = min(min_events, 50)

display_keys = ["misc", "onset", "dur", "inst", "pitch", "antic", "vel"]
key_mapping = {
    "misc": "special_token",
    "onset": "onset",
    "dur": "duration",
    "inst": "instrument",
    "pitch": "pitch",
    "antic": "anticipated",
    "vel": "velocity"
}

columns = pd.MultiIndex.from_product([events_by_dataset.keys(), display_keys], names=['dataset', 'key'])

table_data = []
for event_idx in range(num_rows):
    row = []
    for dataset_name in events_by_dataset.keys():
        events = events_by_dataset[dataset_name]
        event = events[event_idx]
        for display_key in display_keys:
            actual_key = key_mapping[display_key]
            value = event.get(actual_key, "")
            if hasattr(value, 'item'):
                value = value.item()
            elif hasattr(value, 'tolist'):
                value = value.tolist()
            row.append(value)
    table_data.append(row)

df = pd.DataFrame(table_data, columns=columns)
df.to_csv("events.csv", index=False)
df

dataset id_train                                    id_test               ...  \
key         misc  onset   dur inst pitch  antic vel    misc  onset   dur  ...   
0             AR                                         AR               ...   
1                 20.46  0.71    0    E4  False               7.05  0.06  ...   
2                 21.26  0.09    0    E4  False               7.05  0.03  ...   
3                 21.86  0.67    0    E4  False               7.13   0.0  ...   
4                 21.87  0.49    0    A3  False               7.14  0.15  ...   
5                 21.88  0.44    0   F#3  False               7.16  0.13  ...   
6                 21.91  0.41    0    A2  False               7.26  0.09  ...   
7                 21.92  0.44    0    D2  False               7.27  0.11  ...   
8                 21.92  0.49    0    D3  False               7.37  0.09  ...   
9                 22.81  0.18    0   F#3  False               7.39   0.1  ...   
10                23.04  0.15    0    D4  False               7.47  0.06  ...   
11                23.19  0.12    0    E4  False               7.48  0.13  ...   
12                23.34  0.16    0    A4  False               7.52  0.38  ...   
13                23.48  0.35    0    A5  False               7.58  0.42  ...   
14                 23.6  0.19    0    D5  False               8.41  0.15  ...   
15                23.77  0.09    0    E5  False               8.42  0.12  ...   
16                23.99  1.11    0   F#4  False               8.52   0.1  ...   
17                24.25  1.16    0    A4  False               8.53  0.08  ...   
18                24.33  0.27    0    D5  False               8.63  0.12  ...   
19                24.49  0.18    0    E5  False               8.63  0.06  ...   
20                24.67  0.17    0    E6  False               8.72  0.13  ...   
21                25.36  0.09    0    D4  False               8.73  0.14  ...   
22                26.47  1.43    0    B4  False               8.82   0.1  ...   
23                26.64  0.25    0    D5  False               8.83  0.07  ...   
24                26.86  0.68    0    E5  False               8.91  0.13  ...   
25                27.18  0.39    0    G2  False               8.94  0.14  ...   
26                 27.2  0.32    0    C2  False               9.03   0.1  ...   
27                 27.2  0.49    0    E3  False               9.04   0.1  ...   
28                27.49  0.23    0    E4  False               9.14   0.1  ...   
29                27.77  0.19    0    E5  False               9.16  0.09  ...   
30                28.12  0.64    0    B3  False               9.23  0.05  ...   
31                28.42  0.16    0    E4  False               9.24  0.13  ...   
32                28.61  0.18    0    G3  False               9.35  0.24  ...   
33                28.76  0.09    0    E4  False               9.35  0.25  ...   
34                28.87  0.15    0    D5  False               9.58  0.12  ...   
35                29.04  0.16    0    B5  False                9.6  0.06  ...   
36                29.14  0.51    0    D5  False               9.68  0.12  ...   
37                29.24  0.13    0    B4  False               9.69  0.09  ...   
38                29.34  0.19    0    A5  False               9.78  0.11  ...   
39                29.42  0.83    0    G4  False                9.8   0.1  ...   
40                29.51  0.15    0    B4  False              10.07  0.07  ...   

dataset                  ood_test                                     
key     pitch  antic vel     misc  onset   dur inst pitch  antic vel  
0                              AR                                     
1         A#1  False               50.87  3.42    0    C5  False      
2         A#4  False               51.56  2.75    0    C6  False      
3          B1  False                51.6  3.48    0   G#3  False      
4          C5  False                51.6  3.21    0   D#4  False      
5          C2  Fals

## Running all multivariate tests

In [3]:
from data_analysis.mardia import mardia
from data_analysis.royston import royston
from data_analysis.hz import hz
from extract_layers.pooling_functions import pool_mean_std

num_layers = 24

tests = [mardia, royston, hz]

pooling_function = pool_mean_std

## Extract layers

### Sanity check

On an actually multivariate normal distribution, all tests should return a p-value of close to 1.

In [4]:
import numpy as np

X = np.random.randn(4000, 100)
metrics = {}
for test in tests:
    metrics = metrics | test(X)

print(metrics)




  Subsampling to 1000 samples and 100 features for testing
p_value <0.001
{'skewness': np.float64(171855.397), 'kurtosis': np.float64(-2.301), 'skewness_p': 0.395, 'kurtosis_p': 0.021, 'royston_p': 0.36, 'hz_p_value': 0.001}


### Extract layers

In [5]:
from extract_layers.extract_layers_main import extract_representations, collate_fn
from constants.model_constants import JORDAN_MODEL_NAME, DEVICE
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(JORDAN_MODEL_NAME).to(DEVICE)
num_layers = 24

id_train_dataloader = DataLoader(
    id_train_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn
)

extract_representations(
    model=model,
    data=id_train_dataloader,
    pooling_function=pooling_function,
    layers=list(range(num_layers + 1)),
)

/afs/csail.mit.edu/u/j/joeltjy1/ood-detection/src/utils/data_loading.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ex["input_ids"], dtype=torch.long) for ex in examples]
Extracting hidden states: 100%|██████████| 5578/5578 [02:35<00:00, 35.79it/s]


{0: array([[-5.9110806e-03,  6.3979891e-03, -6.0609970e-03, ...,
          3.4708261e-02,  3.5237722e-02,  3.5380285e-02],
        [-7.3599704e-03,  8.1306314e-03, -2.4221973e-03, ...,
          3.3040121e-02,  3.6365788e-02,  3.8337175e-02],
        [-5.6966753e-03,  8.0154659e-03, -1.6049905e-03, ...,
          3.3712570e-02,  3.4965046e-02,  4.0272471e-02],
        ...,
        [ 9.2563396e-03,  1.7316611e-03, -3.5579493e-03, ...,
          3.0467726e-02,  3.5387058e-02,  3.7506565e-02],
        [ 5.6256950e-03, -6.0085522e-04,  1.4028049e-03, ...,
          3.2196917e-02,  3.4391984e-02,  3.7280243e-02],
        [ 8.1770848e-03,  8.2212966e-05,  4.8882253e-03, ...,
          3.6085494e-02,  3.2617692e-02,  3.5533369e-02]],
       shape=(89238, 2048), dtype=float32),
 1: array([[ 0.00034859,  0.07378831,  0.00954882, ...,  0.08258595,
          0.09581736,  0.135085  ],
        [ 0.01209578,  0.04203761,  0.02592461, ...,  0.08891829,
          0.08475216,  0.12993735],
        [-0.

In [ ]:
import numpy as np
import pandas as pd
from constants.file_format import get_extract_layers_file_path
from tqdm import tqdm


metrics = []

for layer_idx in tqdm(range(num_layers+1), desc="Processing layers"):
    layer_dir = get_extract_layers_file_path(
        dataset_name="id_train_dataset",
        pooling_function_name=pooling_function.__name__,
        layer_idx=layer_idx,
    )
    layer_data = np.load(layer_dir)
    print("Processing layer", layer_idx)
    print("Layer data shape:", layer_data.shape)
    layer_metrics = {}
    for test in tests:
        print("Running test", test.__name__)
        layer_metrics = layer_metrics | test(layer_data)
    metrics.append(layer_metrics)

all_metrics = pd.DataFrame(metrics)


Processing layers:   0%|          | 0/25 [00:00<?, ?it/s]

Processing layer 0
Layer data shape: (89238, 2048)
Running test mardia
  Subsampling to 5000 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 5000 samples and 100 features for testing
p_value <0.001


Processing layers:   4%|▍         | 1/25 [00:04<01:41,  4.24s/it]

Processing layer 1
Layer data shape: (89238, 2048)
Running test mardia
  Subsampling to 5000 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 5000 samples and 100 features for testing


Processing layers:   8%|▊         | 2/25 [00:08<01:42,  4.46s/it]

...

Processing layers:  92%|█████████▏| 23/25 [01:29<00:07,  3.82s/it]

Processing layer 23
Layer data shape: (89238, 2048)
Running test mardia
  Subsampling to 5000 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 5000 samples and 100 features for testing


Processing layers:  96%|█████████▌| 24/25 [01:33<00:03,  3.87s/it]

p_value <0.001
Processing layer 24
Layer data shape: (89238, 2048)
Running test mardia
  Subsampling to 5000 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 5000 samples and 100 features for testing
p_value <0.001


Processing layers: 100%|██████████| 25/25 [01:37<00:00,  3.90s/it]


In [7]:
all_metrics

,skewness,kurtosis,skewness_p,kurtosis_p,royston_p,hz_p_value
0,371099.391,81.283,0.001,0.001,0.001,0.001
1,391726.546,107.861,0.001,0.001,0.001,0.001
2,425042.827,92.074,0.001,0.001,0.001,0.001
3,448607.468,78.716,0.001,0.001,0.001,0.001
4,448737.499,80.867,0.001,0.001,0.001,0.001
5,448853.015,75.688,0.001,0.001,0.001,0.001
6,461434.768,86.290,0.001,0.001,0.001,0.001
7,443254.151,73.873,0.001,0.001,0.001,0.001
8,408421.277,63.187,0.001,0.001,0.001,0.001
9,434861.668,57.497,0.001,0.001,0.001,0.001


In [ ]:

## optionally save it
all_metrics.to_csv("all_metrics.csv", index=False)

: 